In [1]:
import torch
torch.__version__

'1.3.0+cu100'

In [2]:
class StubLogger(object):
    def __getattr__(self, name):
        return self.log_print

    def log_print(self, msg, *args):
        print(msg % args)

LOGGER = StubLogger()
LOGGER.info("Hello %s!", "world")

Hello world!


## DeepCTR codes

### deepctr/layers/core.py

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DNN(nn.Module):
    """The Multi Layer Percetron

      Input shape
        - nD tensor with shape: ``(batch_size, ..., input_dim)``. The most common situation would be a 2D input with shape ``(batch_size, input_dim)``.

      Output shape
        - nD tensor with shape: ``(batch_size, ..., hidden_size[-1])``. For instance, for a 2D input with shape ``(batch_size, input_dim)``, the output would have shape ``(batch_size, hidden_size[-1])``.

      Arguments
        - **inputs_dim**: input feature dimension.

        - **hidden_units**:list of positive integer, the layer number and units in each layer.

        - **activation**: Activation function to use.

        - **l2_reg**: float between 0 and 1. L2 regularizer strength applied to the kernel weights matrix.

        - **dropout_rate**: float in [0,1). Fraction of the units to dropout.

        - **use_bn**: bool. Whether use BatchNormalization before activation or not.

        - **seed**: A Python integer to use as random seed.
    """

    def __init__(self, inputs_dim, hidden_units, activation=F.relu, l2_reg=0, dropout_rate=0, use_bn=False,
                 init_std=0.0001, seed=1024, device='cpu'):
        super(DNN, self).__init__()
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate)
        self.seed = seed
        self.l2_reg = l2_reg
        self.use_bn = use_bn
        if len(hidden_units) == 0:
            raise ValueError("hidden_units is empty!!")
        hidden_units = [inputs_dim] + list(hidden_units)

        self.linears = nn.ModuleList(
            [nn.Linear(hidden_units[i], hidden_units[i + 1]) for i in range(len(hidden_units) - 1)])

        if self.use_bn:
            self.bn = nn.ModuleList(
                [nn.BatchNorm1d(hidden_units[i + 1]) for i in range(len(hidden_units) - 1)])
        for name, tensor in self.linears.named_parameters():
            if 'weight' in name:
                nn.init.normal_(tensor, mean=0, std=init_std)

        self.to(device)

    def forward(self, inputs):
        deep_input = inputs

        for i in range(len(self.linears)):

            fc = self.linears[i](deep_input)

            if self.use_bn:
                fc = self.bn[i](fc)

            fc = self.activation(fc)

            fc = self.dropout(fc)
            deep_input = fc
        return deep_input


class PredictionLayer(nn.Module):
    """
      Arguments
         - **task**: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
         - **use_bias**: bool.Whether add bias term or not.
    """

    def __init__(self, task='binary', use_bias=True, **kwargs):
        if task not in ["binary", "multiclass", "regression"]:
            raise ValueError("task must be binary,multiclass or regression")

        super(PredictionLayer, self).__init__()
        self.use_bias = use_bias
        self.task = task
        if self.use_bias:
            self.bias = nn.Parameter(torch.zeros((1,)))

    def forward(self, X):
        output = X
        if self.use_bias:
            output += self.bias
        if self.task == "binary":
            output = torch.sigmoid(output)
        return output


### deepctr/layers/interaction.py

In [4]:
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F

class FM(nn.Module):
    """Factorization Machine models pairwise (order-2) feature interactions
     without linear term and bias.
      Input shape
        - 3D tensor with shape: ``(batch_size,field_size,embedding_size)``.
      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.
      References
        - [Factorization Machines](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)
    """

    def __init__(self):
        super(FM, self).__init__()

    def forward(self, inputs):
        fm_input = inputs

        square_of_sum = torch.pow(torch.sum(fm_input, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(fm_input * fm_input, dim=1, keepdim=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * torch.sum(cross_term, dim=2, keepdim=False)

        return cross_term


class BiInteractionPooling(nn.Module):
    """Bi-Interaction Layer used in Neural FM,compress the
     pairwise element-wise product of features into one single vector.

      Input shape
        - A 3D tensor with shape:``(batch_size,field_size,embedding_size)``.

      Output shape
        - 3D tensor with shape: ``(batch_size,1,embedding_size)``.

      References
        - [He X, Chua T S. Neural factorization machines for sparse predictive analytics[C]//Proceedings of the 40th International ACM SIGIR conference on Research and Development in Information Retrieval. ACM, 2017: 355-364.](http://arxiv.org/abs/1708.05027)
    """

    def __init__(self):
        super(BiInteractionPooling, self).__init__()

    def forward(self, inputs):
        concated_embeds_value = inputs
        square_of_sum = torch.pow(
            torch.sum(concated_embeds_value, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(
            concated_embeds_value * concated_embeds_value, dim=1, keepdim=True)
        cross_term = 0.5 * (square_of_sum - sum_of_square)
        return cross_term


class AFMLayer(nn.Module):
    """Attentonal Factorization Machine models pairwise (order-2) feature
    interactions without linear term and bias.
      Input shape
        - A list of 3D tensor with shape: ``(batch_size,1,embedding_size)``.
      Output shape
        - 2D tensor with shape: ``(batch_size, 1)``.
      Arguments
        - **in_features** : Positive integer, dimensionality of input features.
        - **attention_factor** : Positive integer, dimensionality of the
         attention network output space.
        - **l2_reg_w** : float between 0 and 1. L2 regularizer strength
         applied to attention network.
        - **dropout_rate** : float between in [0,1). Fraction of the attention net output units to dropout.
        - **seed** : A Python integer to use as random seed.
      References
        - [Attentional Factorization Machines : Learning the Weight of Feature
        Interactions via Attention Networks](https://arxiv.org/pdf/1708.04617.pdf)
    """

    def __init__(self, in_features, attention_factor=4, l2_reg_w=0, dropout_rate=0, seed=1024, device='cpu'):
        super(AFMLayer, self).__init__()
        self.attention_factor = attention_factor
        self.l2_reg_w = l2_reg_w
        self.dropout_rate = dropout_rate
        self.seed = seed
        embedding_size = in_features

        self.attention_W = nn.Parameter(torch.Tensor(
            embedding_size, self.attention_factor))

        self.attention_b = nn.Parameter(torch.Tensor(self.attention_factor))

        self.projection_h = nn.Parameter(
            torch.Tensor(self.attention_factor, 1))

        self.projection_p = nn.Parameter(torch.Tensor(embedding_size, 1))

        for tensor in [self.attention_W, self.projection_h, self.projection_p]:
            nn.init.xavier_normal_(tensor, )

        self.dropout = nn.Dropout(dropout_rate)

        self.to(device)

    def forward(self, inputs):
        embeds_vec_list = inputs
        row = []
        col = []

        for r, c in itertools.combinations(embeds_vec_list, 2):
            row.append(r)
            col.append(c)

        p = torch.cat(row, dim=1)
        q = torch.cat(col, dim=1)
        inner_product = p * q

        bi_interaction = inner_product
        attention_temp = F.relu(torch.tensordot(
            bi_interaction, self.attention_W, dims=([-1], [0])) + self.attention_b)

        self.normalized_att_score = F.softmax(torch.tensordot(
            attention_temp, self.projection_h, dims=([-1], [0])), dim=1)
        attention_output = torch.sum(
            self.normalized_att_score * bi_interaction, dim=1)

        attention_output = self.dropout(attention_output)  # training

        afm_out = torch.tensordot(
            attention_output, self.projection_p, dims=([-1], [0]))
        return afm_out


### deepctr/layers/utils.py

In [5]:
import numpy as np
import torch


def concat_fun(inputs, axis=-1):
    if len(inputs) == 1:
        return inputs[0]
    else:
        return torch.cat(inputs, dim=axis)


def slice_arrays(arrays, start=None, stop=None):
    """Slice an array or list of arrays.

    This takes an array-like, or a list of
    array-likes, and outputs:
        - arrays[start:stop] if `arrays` is an array-like
        - [x[start:stop] for x in arrays] if `arrays` is a list

    Can also work on list/array of indices: `slice_arrays(x, indices)`

    Arguments:
        arrays: Single array or list of arrays.
        start: can be an integer index (start index)
            or a list/array of indices
        stop: integer (stop index); should be None if
            `start` was a list.

    Returns:
        A slice of the array(s).

    Raises:
        ValueError: If the value of start is a list and stop is not None.
    """

    if arrays is None:
        return [None]

    if isinstance(arrays, np.ndarray):
        arrays = [arrays]

    if isinstance(start, list) and stop is not None:
        raise ValueError('The stop argument has to be None if the value of start '
                         'is a list.')
    elif isinstance(arrays, list):
        if hasattr(start, '__len__'):
            # hdf5 datasets only support list objects as indices
            if hasattr(start, 'shape'):
                start = start.tolist()
            return [None if x is None else x[start] for x in arrays]
        else:
            if len(arrays) == 1:
                return arrays[0][start:stop]
            return [None if x is None else x[start:stop] for x in arrays]
    else:
        if hasattr(start, '__len__'):
            if hasattr(start, 'shape'):
                start = start.tolist()
            return arrays[start]
        elif hasattr(start, '__getitem__'):
            return arrays[start:stop]
        else:
            return [None]


### deepctr/inputs.py

In [6]:
import collections
import itertools

import torch

class SparseFeat(collections.namedtuple('SparseFeat', ['name', 'dimension', 'use_hash', 'dtype', 'embedding_name', 'embedding'])):
    __slots__ = ()

    def __new__(cls, name, dimension, use_hash=False, dtype="int32", embedding_name=None, embedding=True):
        if embedding and embedding_name is None:
            embedding_name = name
        return super(SparseFeat, cls).__new__(cls, name, dimension, use_hash, dtype, embedding_name, embedding)


class DenseFeat(collections.namedtuple('DenseFeat', ['name', 'dimension', 'dtype'])):
    __slots__ = ()

    def __new__(cls, name, dimension=1, dtype="float32"):
        return super(DenseFeat, cls).__new__(cls, name, dimension, dtype)


class VarLenSparseFeat(collections.namedtuple('VarLenFeat',
                                  ['name', 'dimension', 'maxlen', 'combiner', 'use_hash', 'dtype', 'embedding_name',
                                   'embedding'])):
    __slots__ = ()

    def __new__(cls, name, dimension, maxlen, combiner="mean", use_hash=False, dtype="float32", embedding_name=None,
                embedding=True):
        if embedding_name is None:
            embedding_name = name
        return super(VarLenSparseFeat, cls).__new__(cls, name, dimension, maxlen, combiner, use_hash, dtype,
                                                    embedding_name, embedding)


def get_feature_names(feature_columns):
    features = build_input_features(feature_columns)
    return list(features.keys())


def get_inputs_list(inputs):
    return list(itertools.chain(*list(map(lambda x: x.values(), filter(lambda x: x is not None, inputs)))))


def build_input_features(feature_columns):
    features = collections.OrderedDict()

    start = 0
    for feat in feature_columns:
        feat_name = feat.name
        if feat_name in features:
            continue
        if isinstance(feat, SparseFeat):
            features[feat_name] = (start, start + 1)
            start += 1
        elif isinstance(feat, DenseFeat):
            features[feat_name] = (start, start + feat.dimension)
            start += feat.dimension
        elif isinstance(feat,VarLenSparseFeat):
            features[feat_name] = (start, start + feat.maxlen)
            start += feat.maxlen
        else:
            raise TypeError("Invalid feature column type,got",type(feat))
    return features


def get_dense_input(features, feature_columns):
    dense_feature_columns = list(filter(lambda x: isinstance(
        x, DenseFeat), feature_columns)) if feature_columns else []
    dense_input_list = []
    for fc in dense_feature_columns:
        dense_input_list.append(features[fc.name])
    return dense_input_list


def combined_dnn_input(sparse_embedding_list, dense_value_list):
    if len(sparse_embedding_list) > 0 and len(dense_value_list) > 0:
        sparse_dnn_input = torch.flatten(
            torch.cat(sparse_embedding_list, dim=-1), start_dim=1)
        dense_dnn_input = torch.flatten(
            torch.cat(dense_value_list, dim=-1), start_dim=1)
        return concat_fun([sparse_dnn_input, dense_dnn_input])
    elif len(sparse_embedding_list) > 0:
        return torch.flatten(torch.cat(sparse_embedding_list, dim=-1), start_dim=1)
    elif len(dense_value_list) > 0:
        return torch.flatten(torch.cat(dense_value_list, dim=-1), start_dim=1)
    else:
        raise NotImplementedError


### deepctr/models/basemodel.py

In [7]:
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.metrics import *
from torch.utils.data import DataLoader
from tqdm import tqdm

class Linear(nn.Module):
    def __init__(self, feature_columns, feature_index, init_std=0.0001, device='cpu'):
        super(Linear, self).__init__()
        self.feature_index = feature_index

        self.sparse_feature_columns = list(
            filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if len(feature_columns) else []
        self.dense_feature_columns = list(
            filter(lambda x: isinstance(x, DenseFeat), feature_columns)) if len(feature_columns) else []

        self.embedding_dict = self.create_embedding_matrix(self.sparse_feature_columns, 1, init_std, sparse=False).to(
            device)

        #         nn.ModuleDict(
        #             {feat.embedding_name: nn.Embedding(feat.dimension, 1, sparse=True) for feat in
        #              self.sparse_feature_columns}
        #         )
        # .to("cuda:1")
        for tensor in self.embedding_dict.values():
            nn.init.normal_(tensor.weight, mean=0, std=init_std)

        if len(self.dense_feature_columns) > 0:
            self.weight = nn.Parameter(torch.Tensor(sum(fc.dimension for fc in self.dense_feature_columns), 1)).to(
                device)
            torch.nn.init.normal_(self.weight, mean=0, std=init_std)

    def forward(self, X):

        sparse_embedding_list = [self.embedding_dict[feat.embedding_name](
            X[:, self.feature_index[feat.name][0]:self.feature_index[feat.name][1]].long()) for
            feat in self.sparse_feature_columns]

        dense_value_list = [X[:, self.feature_index[feat.name][0]:self.feature_index[feat.name][1]] for feat in
                            self.dense_feature_columns]

        if len(sparse_embedding_list) > 0 and len(dense_value_list) > 0:
            linear_sparse_logit = torch.sum(
                torch.cat(sparse_embedding_list, dim=-1), dim=-1, keepdim=False)
            linear_dense_logit = torch.cat(
                dense_value_list, dim=-1).matmul(self.weight)
            linear_logit = linear_sparse_logit + linear_dense_logit
        elif len(sparse_embedding_list) > 0:
            linear_logit = torch.sum(
                torch.cat(sparse_embedding_list, dim=-1), dim=-1, keepdim=False)
        elif len(dense_value_list) > 0:
            linear_logit = torch.cat(
                dense_value_list, dim=-1).matmul(self.weight)
        else:
            linear_logit = torch.zeros([X.shape[0], 1])
        return linear_logit

    def create_embedding_matrix(self, feature_columns, embedding_size, init_std=0.0001, sparse=False):

        sparse_feature_columns = list(
            filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if len(feature_columns) else []

        embedding_dict = nn.ModuleDict(
            {feat.embedding_name: nn.Embedding(feat.dimension, embedding_size, sparse=sparse) for feat in
             sparse_feature_columns}
        )
        for tensor in embedding_dict.values():
            nn.init.normal_(tensor.weight, mean=0, std=init_std)

        return embedding_dict


class BaseModel(nn.Module):
    def __init__(self,
                 linear_feature_columns, dnn_feature_columns, embedding_size=8, dnn_hidden_units=(128, 128),
                 l2_reg_linear=1e-5,
                 l2_reg_embedding=1e-5, l2_reg_dnn=0, init_std=0.0001, seed=1024, dnn_dropout=0, dnn_activation='relu',
                 task='binary', device='cpu'):

        super(BaseModel, self).__init__()

        self.reg_loss = torch.zeros((1,), device=device)
        self.device = device  # device

        self.feature_index = build_input_features(
            linear_feature_columns + dnn_feature_columns)
        self.dnn_feature_columns = dnn_feature_columns

        self.embedding_dict = self.create_embedding_matrix(dnn_feature_columns, embedding_size, init_std,
                                                           sparse=False).to(device)
        #         nn.ModuleDict(
        #             {feat.embedding_name: nn.Embedding(feat.dimension, embedding_size, sparse=True) for feat in
        #              self.dnn_feature_columns}
        #         )

        self.linear_model = Linear(
            linear_feature_columns, self.feature_index, device=device)

        self.add_regularization_loss(
            self.embedding_dict.parameters(), l2_reg_embedding)
        self.add_regularization_loss(
            self.linear_model.parameters(), l2_reg_linear)

        self.out = PredictionLayer(task, )
        self.to(device)

    def fit(self, x=None,
            y=None,
            batch_size=None,
            epochs=1,
            verbose=1,
            initial_epoch=0,
            validation_split=0.,
            validation_data=None,
            shuffle=True, ):
        """

        :param x: Numpy array of training data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs).If input layers in the model are named, you can also pass a
            dictionary mapping input names to Numpy arrays.
        :param y: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs).
        :param batch_size: Integer or `None`. Number of samples per gradient update. If unspecified, `batch_size` will default to 256.
        :param epochs: Integer. Number of epochs to train the model. An epoch is an iteration over the entire `x` and `y` data provided. Note that in conjunction with `initial_epoch`, `epochs` is to be understood as "final epoch". The model is not trained for a number of iterations given by `epochs`, but merely until the epoch of index `epochs` is reached.
        :param verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
        :param initial_epoch: Integer. Epoch at which to start training (useful for resuming a previous training run).
        :param validation_split: Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. The validation data is selected from the last samples in the `x` and `y` data provided, before shuffling.
        :param validation_data: tuple `(x_val, y_val)` or tuple `(x_val, y_val, val_sample_weights)` on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. `validation_data` will override `validation_split`.
        :param shuffle: Boolean. Whether to shuffle the order of the batches at the beginning of each epoch.

        """
        if isinstance(x,dict):
            x = [x[feature] for feature in self.feature_index]
        if validation_data:
            if len(validation_data) == 2:
                val_x, val_y = validation_data
                val_sample_weight = None
            elif len(validation_data) == 3:
                val_x, val_y, val_sample_weight = validation_data  # pylint: disable=unpacking-non-sequence
            else:
                raise ValueError(
                    'When passing a `validation_data` argument, '
                    'it must contain either 2 items (x_val, y_val), '
                    'or 3 items (x_val, y_val, val_sample_weights), '
                    'or alternatively it could be a dataset or a '
                    'dataset or a dataset iterator. '
                    'However we received `validation_data=%s`' % validation_data)
            if isinstance(val_x, dict):
                val_x = [val_x[feature] for feature in self.feature_index]

        elif validation_split and 0. < validation_split < 1.:
            if hasattr(x[0], 'shape'):
                split_at = int(x[0].shape[0] * (1. - validation_split))
            else:
                split_at = int(len(x[0]) * (1. - validation_split))
            x, val_x = (slice_arrays(x, 0, split_at),
                        slice_arrays(x, split_at))
            y, val_y = (slice_arrays(y, 0, split_at),
                        slice_arrays(y, split_at))

        else:
            val_x = []
            val_y = []
        for i in range(len(x)):
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        train_tensor_data = Data.TensorDataset(
            torch.from_numpy(
                np.concatenate(x, axis=-1)),
            torch.from_numpy(y))
        if batch_size is None:
            batch_size = 256
        train_loader = DataLoader(
            dataset=train_tensor_data, shuffle=shuffle, batch_size=batch_size)

        print(self.device, end="\n")
        model = self.train()
        loss_func = self.loss_func
        optim = self.optim

        sample_num = len(train_tensor_data)
        steps_per_epoch = (sample_num - 1) // batch_size + 1

        print("Train on {0} samples, validate on {1} samples, {2} steps per epoch".format(
            len(train_tensor_data), len(val_y),steps_per_epoch))
        for epoch in range(initial_epoch, epochs):
            start_time = time.time()
            loss_epoch = 0
            total_loss_epoch = 0
            # if abs(loss_last - loss_now) < 0.0
            train_result = {}
            try:
                with tqdm(enumerate(train_loader), disable=verbose != 1) as t:
                    for index, (x_train, y_train) in t:
                        x = x_train.to(self.device).float()
                        y = y_train.to(self.device).float()

                        y_pred = model(x).squeeze()

                        optim.zero_grad()
                        loss = loss_func(y_pred, y.squeeze(), reduction='sum')

                        total_loss = loss + self.reg_loss

                        loss_epoch += loss.item()
                        total_loss_epoch += total_loss.item()
                        total_loss.backward(retain_graph=True)
                        optim.step()

                        if verbose > 0:
                            for name, metric_fun in self.metrics.items():
                                if name not in train_result:
                                    train_result[name] = []
                                train_result[name].append(metric_fun(
                                    y.cpu().data.numpy(), y_pred.cpu().data.numpy()))

            except KeyboardInterrupt:
                t.close()
                raise
            t.close()

            epoch_time = int(time.time() - start_time)
            if verbose > 0:
                print('Epoch {0}/{1}'.format(epoch + 1, epochs))

                eval_str = "{0}s - loss: {1: .4f}".format(
                    epoch_time, total_loss_epoch / sample_num)

                for name, result in train_result.items():
                    eval_str += " - " + name + \
                        ": {0: .4f}".format(np.sum(result) / steps_per_epoch)

                if len(val_x) and len(val_y):
                    eval_result = self.evaluate(val_x, val_y, batch_size)

                    for name, result in eval_result.items():
                        eval_str += " - val_" + name + \
                            ": {0: .4f}".format(result)
                print(eval_str)

    def evaluate(self, x, y, batch_size=256):
        """

        :param x: Numpy array of test data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs).
        :param y: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs).
        :param batch_size:
        :return: Integer or `None`. Number of samples per evaluation step. If unspecified, `batch_size` will default to 256.
        """
        pred_ans = self.predict(x, batch_size)
        eval_result = {}
        for name, metric_fun in self.metrics.items():
            eval_result[name] = metric_fun(y, pred_ans)
        return eval_result

    def predict(self, x, batch_size=256):
        """

        :param x: The input data, as a Numpy array (or list of Numpy arrays if the model has multiple inputs).
        :param batch_size: Integer. If unspecified, it will default to 256.
        :return: Numpy array(s) of predictions.
        """
        model = self.eval()
        if isinstance(x, dict):
            x = [x[feature] for feature in self.feature_index]
        for i in range(len(x)):
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        tensor_data = Data.TensorDataset(
            torch.from_numpy(np.concatenate(x, axis=-1)))
        test_loader = DataLoader(
            dataset=tensor_data, shuffle=False, batch_size=batch_size)

        pred_ans = []
        with torch.no_grad():
            for index, x_test in enumerate(test_loader):
                x = x_test[0].to(self.device).float()
                # y = y_test.to(self.device).float()

                y_pred = model(x).cpu().data.numpy()  # .squeeze()
                pred_ans.append(y_pred)
        return np.concatenate(pred_ans)

    def input_from_feature_columns(self, X, feature_columns, embedding_dict, support_dense=True):

        sparse_feature_columns = list(
            filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if len(feature_columns) else []
        dense_feature_columns = list(
            filter(lambda x: isinstance(x, DenseFeat), feature_columns)) if len(feature_columns) else []

        varlen_sparse_feature_columns = list(
            filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []

        if not support_dense and len(dense_feature_columns) > 0:
            raise ValueError(
                "DenseFeat is not supported in dnn_feature_columns")

        sparse_embedding_list = [embedding_dict[feat.embedding_name](
            X[:, self.feature_index[feat.name][0]:self.feature_index[feat.name][1]].long()) for
            feat in sparse_feature_columns]
        varlen_sparse_embedding_list = [embedding_dict[feat.embedding_name](
            X[:, self.feature_index[feat.name][0]:self.feature_index[feat.name][1]].long()) for
            feat in varlen_sparse_feature_columns]
        varlen_sparse_embedding_list = list(
            map(lambda x: x.unsqueeze(dim=1), varlen_sparse_embedding_list))

        dense_value_list = [X[:, self.feature_index[feat.name][0]:self.feature_index[feat.name][1]] for feat in
                            dense_feature_columns]

        return sparse_embedding_list + varlen_sparse_embedding_list, dense_value_list

    def create_embedding_matrix(self, feature_columns, embedding_size, init_std=0.0001, sparse=False):

        sparse_feature_columns = list(
            filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if len(feature_columns) else []

        varlen_sparse_feature_columns = list(
            filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if len(feature_columns) else []

        embedding_dict = nn.ModuleDict(
            {feat.embedding_name: nn.Embedding(feat.dimension, embedding_size, sparse=sparse) for feat in
             sparse_feature_columns}
        )

        for feat in varlen_sparse_feature_columns:
            embedding_dict[feat.embedding_name] = nn.EmbeddingBag(
                feat.dimension, embedding_size, sparse=sparse, mode=feat.combiner)

        for tensor in embedding_dict.values():
            nn.init.normal_(tensor.weight, mean=0, std=init_std)

        return embedding_dict

    def compute_input_dim(self, feature_columns, embedding_size=1, include_sparse=True, include_dense=True, feature_group=False):
        sparse_feature_columns = list(
            filter(lambda x: isinstance(x, (SparseFeat, VarLenSparseFeat)), feature_columns)) if len(feature_columns) else []
        dense_feature_columns = list(
            filter(lambda x: isinstance(x, DenseFeat), feature_columns)) if len(feature_columns) else []

        dense_input_dim = sum(
            map(lambda x: x.dimension, dense_feature_columns))
        if feature_group:
            sparse_input_dim = len(sparse_feature_columns)
        else:
            sparse_input_dim = len(sparse_feature_columns) * embedding_size
        input_dim = 0
        if include_sparse:
            input_dim += sparse_input_dim
        if include_dense:
            input_dim += dense_input_dim
        return input_dim

    def add_regularization_loss(self, weight_list, weight_decay, p=2):
        reg_loss = torch.zeros((1,), device=self.device)
        for w in weight_list:
            if isinstance(w, tuple):
                l2_reg = torch.norm(w[1], p=p, )
            else:
                l2_reg = torch.norm(w, p=p, )
            reg_loss = reg_loss + l2_reg
        reg_loss = weight_decay * reg_loss
        self.reg_loss += reg_loss

    def compile(self, optimizer,
                loss=None,
                metrics=None,
                ):
        """
        :param optimizer: String (name of optimizer) or optimizer instance. See [optimizers](https://pytorch.org/docs/stable/optim.html).
        :param loss: String (name of objective function) or objective function. See [losses](https://pytorch.org/docs/stable/nn.functional.html#loss-functions).
        :param metrics: List of metrics to be evaluated by the model during training and testing. Typically you will use `metrics=['accuracy']`.
        """

        self.optim = self._get_optim(optimizer)
        self.loss_func = self._get_loss_func(loss)
        self.metrics = self._get_metrics(metrics)

    def _get_optim(self, optimizer):
        if isinstance(optimizer, str):
            if optimizer == "sgd":
                optim = torch.optim.SGD(self.parameters(), lr=0.01)
            elif optimizer == "adam":
                optim = torch.optim.Adam(self.parameters())  # 0.001
            elif optimizer == "adagrad":
                optim = torch.optim.Adagrad(self.parameters())  # 0.01
            elif optimizer == "rmsprop":
                optim = torch.optim.RMSprop(self.parameters())
            else:
                raise NotImplementedError
        else:
            optim = optimizer
        return optim

    def _get_loss_func(self, loss):
        if isinstance(loss, str):
            if loss == "binary_crossentropy":
                loss_func = F.binary_cross_entropy
            elif loss == "mse":
                loss_func = F.mse_loss
            elif loss == "mae":
                loss_func = F.l1_loss
            else:
                raise NotImplementedError
        else:
            loss_func = loss
        return loss_func

    def _get_metrics(self, metrics):
        metrics_ = {}
        if metrics:
            for metric in metrics:
                if metric == "binary_crossentropy" or metric == "logloss":
                    metrics_[metric] = log_loss
                if metric == "auc":
                    metrics_[metric] = roc_auc_score
                if metric == "mse":
                    metrics_[metric] = mean_squared_error
                if metric == "accuracy" or metric == "acc":
                    metrics_[metric] = lambda y_true, y_pred: accuracy_score(
                        y_true, np.where(y_pred > 0.5, 1, 0))
        return metrics_


### deepctr/models/afm.py

In [8]:
import torch
import torch.nn.functional as F

class AFM(BaseModel):

    """Instantiates the Attentional Factorization Machine architecture.

    :param linear_feature_columns: An iterable containing all the features used by linear part of the model.
    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param embedding_size: positive integer,sparse feature embedding_size
    :param use_attention: bool,whether use attention or not,if set to ``False``.it is the same as **standard Factorization Machine**
    :param attention_factor: positive integer,units in attention net
    :param l2_reg_linear: float. L2 regularizer strength applied to linear part
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param l2_reg_att: float. L2 regularizer strength applied to attention net
    :param afm_dropout: float in [0,1), Fraction of the attention net output units to dropout.
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :param device: str, ``"cpu"`` or ``"cuda:0"``
    :return: A PyTorch model instance.

    """

    def __init__(self,linear_feature_columns, dnn_feature_columns, embedding_size=8, use_attention=True, attention_factor=8,
                 l2_reg_linear=1e-5, l2_reg_embedding=1e-5, l2_reg_att=1e-5, afm_dropout=0, init_std=0.0001, seed=1024,
                 task='binary', device='cpu'):
        super(AFM, self).__init__(linear_feature_columns, dnn_feature_columns, embedding_size=embedding_size,
                                  dnn_hidden_units=[],
                                  l2_reg_linear=l2_reg_linear,
                                  l2_reg_embedding=l2_reg_embedding, l2_reg_dnn=0, init_std=init_std,
                                  seed=seed,
                                  dnn_dropout=0, dnn_activation=F.relu,
                                  task=task, device=device)

        self.use_attention = use_attention

        if use_attention:
            self.fm = AFMLayer(embedding_size, attention_factor, l2_reg_att, afm_dropout,
                               seed, device)
            self.add_regularization_loss(self.fm.attention_W, l2_reg_att)
        else:
            self.fm = FM()

        self.to(device)

    def forward(self, X):

        sparse_embedding_list, _ = self.input_from_feature_columns(X, self.dnn_feature_columns,
                                                                                  self.embedding_dict,support_dense=False)
        logit = self.linear_model(X)
        if len(sparse_embedding_list) > 0:
            if self.use_attention:
                logit += self.fm(sparse_embedding_list)
            else:
                logit += self.fm(torch.cat(sparse_embedding_list, dim=1))

        y_pred = self.out(logit)

        return y_pred


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NFM(BaseModel):
    """Instantiates the NFM Network architecture.

    :param linear_feature_columns: An iterable containing all the features used by linear part of the model.
    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param embedding_size: positive integer,sparse feature embedding_size
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of deep net
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param l2_reg_linear: float. L2 regularizer strength applied to linear part.
    :param l2_reg_dnn: float . L2 regularizer strength applied to DNN
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param biout_dropout: When not ``None``, the probability we will drop out the output of BiInteractionPooling Layer.
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param dnn_activation: Activation function to use in deep net
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :param device: str, ``"cpu"`` or ``"cuda:0"``
    :return: A PyTorch model instance.
    
    """

    def __init__(self,
                 linear_feature_columns, dnn_feature_columns, embedding_size=8, dnn_hidden_units=(128, 128),
                 l2_reg_embedding=1e-5, l2_reg_linear=1e-5, l2_reg_dnn=0, init_std=0.0001, seed=1024, bi_dropout=0,
                 dnn_dropout=0, dnn_activation=F.relu, task='binary', device='cpu'):
        super(NFM, self).__init__(linear_feature_columns, dnn_feature_columns, embedding_size=embedding_size,
                                  dnn_hidden_units=dnn_hidden_units,
                                  l2_reg_linear=l2_reg_linear,
                                  l2_reg_embedding=l2_reg_embedding, l2_reg_dnn=l2_reg_dnn, init_std=init_std,
                                  seed=seed,
                                  dnn_dropout=dnn_dropout, dnn_activation=dnn_activation,
                                  task=task, device=device)

        self.dnn = DNN(self.compute_input_dim(dnn_feature_columns, embedding_size, include_sparse=False) + embedding_size,
                       dnn_hidden_units,
                       activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=False,
                       init_std=init_std, device=device)
        self.dnn_linear = nn.Linear(
            dnn_hidden_units[-1], 1, bias=False).to(device)
        self.add_regularization_loss(
            filter(lambda x: 'weight' in x[0] and 'bn' not in x[0], self.dnn.named_parameters()), l2_reg_dnn)
        self.add_regularization_loss(self.dnn_linear.weight, l2_reg_dnn)
        self.bi_pooling = BiInteractionPooling()
        self.bi_dropout = bi_dropout
        if self.bi_dropout > 0:
            self.dropout = nn.Dropout(bi_dropout)
        self.to(device)

    def forward(self, X):

        sparse_embedding_list, dense_value_list = self.input_from_feature_columns(X, self.dnn_feature_columns,
                                                                                  self.embedding_dict)
        linear_logit = self.linear_model(X)
        fm_input = torch.cat(sparse_embedding_list, dim=1)
        bi_out = self.bi_pooling(fm_input)
        if self.bi_dropout:
            bi_out = self.dropout(bi_out)

        dnn_input = combined_dnn_input([bi_out], dense_value_list)
        dnn_output = self.dnn(dnn_input)
        dnn_logit = self.dnn_linear(dnn_output)

        logit = linear_logit + dnn_logit

        y_pred = self.out(logit)

        return y_pred


## Learning methods

In [10]:
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from tqdm import tqdm

def np_sigmoid(x):
    z = np.where(x >= 0, np.exp(-x), np.exp(x))
    nom = np.where(x >= 0, 1, z)
    return nom / (1 + z)


### FIT MODEL WITH PAIR-WISE LOSS FUNC AND DATA ###
def regression_pairwise_loss(positive_pred, negative_pred, reduction="mean"):
    pred_diff = torch.clamp(positive_pred - negative_pred, -80.0, 1e8)
    ret = (pred_diff - 1) ** 2
    if reduction == "mean":
        ret = torch.mean(ret)
    elif reduction == "sum":
        ret = torch.sum(ret)
    return ret


def bayesian_pairwise_ranking_loss(positive_pred, negative_pred, reduction="mean"):
    pred_diff = torch.clamp(positive_pred - negative_pred, -80.0, 1e8)
    # The naive style BPR is -torch.log(torch.sigmoid(positive_pred - negative_pred))
    # or just torch.nn.functional.logsigmoid(positive_pred - negative_pred)
    #  (like the implementation of CFM).
    # Move the initial negative sign into the log function, reversing the sigmoid expression,
    # the less-better style becomes torch.log(1 + torch.exp(negative_pred - positive_pred))
    #  (like the commented implementation in APR).
    # Due to the numerically unstability of the style above, the implementation APR
    # suggests using torch.nn.functional.softplus instead.
    ret = F.softplus(-pred_diff)
    if reduction == "mean":
        ret = torch.mean(ret)
    elif reduction == "sum":
        ret = torch.sum(ret)
    return ret


def pairwise_fit(model, pairwise_training_data, *, loss_func,
                 batch_size=256, epochs=1, initial_epoch=0,
                 validation_split=0., validation_data=None,
                 shuffle=True, verbose=1, apply_sigmoid=True):
    """
    :param positive_x: Numpy array of training data (if the model has a single
        input), or list of Numpy arrays (if the model has multiple inputs). If
        input layers in the model are named, you can also pass a dictionary
        mapping input names to Numpy arrays.
    :param negative_x: Numpy array of training data (if the model has a single
        input), or list of Numpy arrays (if the model has multiple inputs). If
        input layers in the model are named, you can also pass a dictionary
        mapping input names to Numpy arrays.
    :param y: Numpy array of target (label) data (if the model has a single
        output), or list of Numpy arrays (if the model has multiple outputs).
    :param batch_size: Integer or `None`. Number of samples per gradient update. If unspecified, `batch_size` will default to 256.
    :param epochs: Integer. Number of epochs to train the model. An epoch is an iteration over the entire `x` and `y` data provided. Note that in conjunction with `initial_epoch`, `epochs` is to be understood as "final epoch". The model is not trained for a number of iterations given by `epochs`, but merely until the epoch of index `epochs` is reached.
    :param verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
    :param initial_epoch: Integer. Epoch at which to start training (useful for resuming a previous training run).
    :param validation_split: Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. The validation data is selected from the last samples in the `x` and `y` data provided, before shuffling.
    :param validation_data: tuple `(x_val, y_val)` or tuple `(x_val, y_val, val_sample_weights)` on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. `validation_data` will override `validation_split`.
    :param shuffle: Boolean. Whether to shuffle the order of the batches at the beginning of each epoch.
    """
    positive_x = pairwise_training_data.positive_x
    y = pairwise_training_data.y

    if isinstance(positive_x, dict):
        positive_x = [positive_x[feature] for feature in model.feature_index]

    if validation_data:
        if len(validation_data) == 2:
            val_x, val_y = validation_data
            val_sample_weight = None
        elif len(validation_data) == 3:
            val_x, val_y, val_sample_weight = validation_data  # pylint: disable=unpacking-non-sequence
        else:
            raise ValueError(
                'When passing a `validation_data` argument, '
                'it must contain either 2 items (x_val, y_val), '
                'or 3 items (x_val, y_val, val_sample_weights), '
                'or alternatively it could be a dataset or a '
                'dataset or a dataset iterator. '
                'However we received `validation_data=%s`' % validation_data)

        if isinstance(val_x, dict):
            val_x = [val_x[feature] for feature in model.feature_index]
    elif validation_split and 0. < validation_split < 1.:
        if hasattr(positive_x[0], 'shape'):
            split_at = int(positive_x[0].shape[0] * (1. - validation_split))
        else:
            split_at = int(len(positive_x[0]) * (1. - validation_split))

        positive_x, val_x = (slice_arrays(positive_x, 0, split_at),
                             slice_arrays(positive_x, split_at))
        y, val_y = (slice_arrays(y, 0, split_at),
                    slice_arrays(y, split_at))
    else:
        val_x = []
        val_y = []

    for i in range(len(positive_x)):
        if len(positive_x[i].shape) == 1:
            positive_x[i] = np.expand_dims(positive_x[i], axis=1)

    LOGGER.info(model.device)
    model = model.train()
    optim = model.optim

    sample_num = len(positive_x[0])
    steps_per_epoch = (sample_num - 1) // batch_size + 1

    LOGGER.info("Train on {0} samples, validate on {1} samples, {2} steps per epoch".format(
          sample_num, len(val_y), steps_per_epoch))
    for epoch in range(initial_epoch, epochs):
        start_time = time.time()

        #@Prepare negative samples
        pairwise_training_data.resample_negatives()
        negative_x = pairwise_training_data.negative_x

        if isinstance(negative_x, dict):
            negative_x = [negative_x[feature] for feature in model.feature_index]

        # Throw out corresponding negative samples for split validation positive samples
        if validation_split and 0. < validation_split < 1.:
            negative_x = slice_arrays(negative_x, 0, split_at)

        for i in range(len(negative_x)):
            if len(negative_x[i].shape) == 1:
                negative_x[i] = np.expand_dims(negative_x[i], axis=1)

        #@Assemble training data of epoch
        train_tensor_data = torch.utils.data.TensorDataset(
            torch.from_numpy(np.concatenate(positive_x, axis=-1)),
            torch.from_numpy(np.concatenate(negative_x, axis=-1)),
            torch.from_numpy(y)
        )
        train_loader = torch.utils.data.DataLoader(dataset=train_tensor_data,
                                                   shuffle=shuffle, batch_size=batch_size)

        #@Fitting model with assembled data 
        loss_epoch = 0
        total_loss_epoch = 0
        # if abs(loss_last - loss_now) < 0.0
        train_result = {}
        with tqdm(enumerate(train_loader), disable=(verbose != 1)) as t:
            for _, (positive_x_train, negative_x_train, y_train) in t:
                placed_positive_x = positive_x_train.to(model.device).float()
                placed_negative_x = negative_x_train.to(model.device).float()

                positive_y_pred = model(placed_positive_x).squeeze(dim=1)
                negative_y_pred = model(placed_negative_x).squeeze(dim=1)

                optim.zero_grad()
                loss = loss_func(positive_y_pred, negative_y_pred, reduction='sum')

                total_loss = loss + model.reg_loss

                loss_epoch += loss.item()
                total_loss_epoch += total_loss.item()
                total_loss.backward(retain_graph=True)
                optim.step()

                if verbose > 0:
                    ndarray_y = y_train.cpu().data.numpy()
                    ndarray_positive_y_pred = positive_y_pred.cpu().data.numpy()
                    if apply_sigmoid:
                        ndarray_positive_y_pred = np_sigmoid(ndarray_positive_y_pred)
                    for name, metric_fun in model.metrics.items():
                        if name not in train_result:
                            train_result[name] = []
                        train_result[name].append(metric_fun(ndarray_y, ndarray_positive_y_pred))

        epoch_time = int(time.time() - start_time)
        if verbose > 0:
            LOGGER.info('Epoch {0}/{1}'.format(epoch + 1, epochs))

            eval_str = "{0}s - loss: {1: .4f}".format(epoch_time, total_loss_epoch / sample_num)

            for name, result in train_result.items():
                eval_str += " - {0}: {1: .4f}".format(name, np.sum(result) / steps_per_epoch)

            if len(val_x) and len(val_y):
                eval_result = pairwise_evaluate(model, val_x, val_y, batch_size,
                                                apply_sigmoid=apply_sigmoid)
                for name, result in eval_result.items():
                    eval_str += " - val_{0}: {1: .4f}".format(name, result)
            LOGGER.info(eval_str)


def pairwise_evaluate(model, x, y, batch_size=256, *,
                      apply_sigmoid=True):
    """

    :param x: Numpy array of test data (if the model has a single input), or list of Numpy arrays (if the model has multiple inputs).
    :param y: Numpy array of target (label) data (if the model has a single output), or list of Numpy arrays (if the model has multiple outputs).
    :param batch_size:
    :return: Integer or `None`. Number of samples per evaluation step. If unspecified, `batch_size` will default to 256.
    """
    pred_ans = model.predict(x, batch_size)
    if apply_sigmoid:
        pred_ans = np_sigmoid(pred_ans)
    eval_result = {}
    for name, metric_fun in model.metrics.items():
        eval_result[name] = metric_fun(y, pred_ans)
    return eval_result


## Data Loading and Recommending Codes

In [11]:
import collections

import numpy as np
import pandas as pd

class DatasetBase(object):
    __slots__ = ("train_feature_file", "test_feature_file",
                 "user_feature_columns", "user_column", "item_feature_columns", "item_column",
                 "train_feature_data", "test_feature_data",
                 "feature_counts", "features", "user_feature_data", "item_feature_data",
                 "num_queries",
                 "train_responses", "test_responses", "test_query_ids", "candidate_item_ids",)

    def __init__(self, train_feature_file, test_feature_file, *,
                 user_feature_columns, user_column, item_feature_columns, item_column):
        self.train_feature_file = train_feature_file
        self.test_feature_file = test_feature_file
        self.user_feature_columns = user_feature_columns
        self.user_column = user_column
        self.item_feature_columns = item_feature_columns
        self.item_column = item_column

        self.train_feature_data = pd.read_csv(self.train_feature_file, sep='\t')
        self.test_feature_data = pd.read_csv(self.test_feature_file, sep='\t')
        feature_data = pd.concat([self.train_feature_data, self.test_feature_data])
        
        self.feature_counts = dict()
        for feature_column in self.user_feature_columns:
            self.feature_counts[feature_column] = feature_data[feature_column].max() + 1
        for feature_column in self.item_feature_columns:
            self.feature_counts[feature_column] = feature_data[feature_column].max() + 1
        
        self.features = collections.OrderedDict()
        for feature_column in self.user_feature_columns:
            self.features[feature_column] = SparseFeat(feature_column, self.feature_counts[feature_column])
        for feature_column in self.item_feature_columns:
            self.features[feature_column] = SparseFeat(feature_column, self.feature_counts[feature_column])
        
        self.user_feature_data = feature_data[self.user_feature_columns].drop_duplicates()\
            .sort_values(by=self.user_feature_columns)
        self.item_feature_data = feature_data[self.item_feature_columns].drop_duplicates()\
            .sort_values(by=self.item_column)
        
        self.num_queries = self.user_feature_data.shape[0]
        self.user_feature_data["query_id"] = np.arange(self.num_queries)
        self.train_responses = pd.merge(self.train_feature_data, self.user_feature_data,
                                        how="left", on=self.user_feature_columns)[["query_id", self.item_column]]
        self.test_responses = pd.merge(self.test_feature_data, self.user_feature_data,
                                        how="left", on=self.user_feature_columns)[["query_id", self.item_column]]
        self.test_query_ids = self.test_responses["query_id"].drop_duplicates()\
            .sort_values().reset_index(drop=True)
        self.candidate_item_ids = self.test_responses[self.item_column].drop_duplicates()\
            .sort_values().reset_index(drop=True)

        self.user_feature_data.set_index("query_id", drop=False, inplace=True)
        self.item_feature_data.set_index(self.item_column, drop=False, inplace=True)
    
    def compile_feeds(self, query_ids, item_ids, batch_size=None):
        if np.isscalar(query_ids):
            query_ids = np.full(len(item_ids), query_ids)
        else:
            query_ids = np.array(query_ids).reshape((len(item_ids),))
        user_feeds = self.user_feature_data.loc[query_ids, self.user_feature_columns].reset_index(drop=True)
        item_feeds = self.item_feature_data.loc[item_ids, self.item_feature_columns].reset_index(drop=True)
        raw_feeds = pd.concat([user_feeds, item_feeds], axis="columns")
        feeds = {c: raw_feeds[c] for c in self.features.keys()}
        return feeds


class FrappeData(DatasetBase):
    USER_FEATURE_COLUMNS = ["user", "daytime", "weekday", "isweekend", "homework", "weather", "country", "city"]
    USER_COLUMN = "user"
    ITEM_FEATURE_COLUMNS = ["item", "cost"]
    ITEM_COLUMN = "item"

    __slots__ = ()

    def __init__(self, train_feature_file, test_feature_file):
        super(FrappeData, self).__init__(train_feature_file, test_feature_file,
                                         user_feature_columns=self.USER_FEATURE_COLUMNS,
                                         user_column=self.USER_COLUMN,
                                         item_feature_columns=self.ITEM_FEATURE_COLUMNS,
                                         item_column=self.ITEM_COLUMN)


### Dataset wrappers

In [12]:
import numpy as np
import scipy.sparse
import sklearn.utils

class BatchData(object):
    __slots__ = ("X", "Y")

    def __init__(self, **kwargs):
        for slot in self.__slots__:
            setattr(self, slot, kwargs[slot])


class PointwiseData(object):
    def __init__(self, base, num_negatives=2, negative_value=-1.0, random_state=None,
                 batch_size=None):
        LOGGER.critical("PointwiseData revision 20.1.7")
        self._base = base
        self._batch_size = batch_size

        train_responses = valid_responses = self._base.train_responses.assign(value=1)
        if num_negatives > 0:
            train_responses = self._tile_negatives(train_responses, num_negatives,
                                                   negative_value, random_state)

        self.train_data = self._compile_dataset(train_responses["query_id"].values,
                                                train_responses[self._base.item_column].values,
                                                train_responses["value"].values,
                                                batch_size=self._batch_size)
        self.valid_data = self._compile_dataset(valid_responses["query_id"].values,
                                                valid_responses[self._base.item_column].values,
                                                valid_responses["value"].values,
                                                batch_size=self._batch_size)
        # self.Test_data = self._compile_dataset(self._base.user_event_tests["user_id"].values,
        #                                        self._base.user_event_tests["event_id"].values,
        #                                        np.ones(self._base.user_event_tests.shape[0]))

    def _tile_negatives(self, train_responses, num_negatives, negative_value, random_state):
        rand_state = sklearn.utils.check_random_state(random_state)
        negative_candidates = np.setdiff1d(np.arange(self._base.feature_counts[self._base.item_column]),
                                           self._base.candidate_item_ids.values)
        used = scipy.sparse.csr_matrix((train_responses["value"].values,
                                        (train_responses["query_id"],
                                         train_responses[self._base.item_column])),
                                       shape=(self._base.num_queries,
                                              self._base.feature_counts[self._base.item_column])).todok()
        user_ids = list()
        item_ids = list()
        values = list()
        for row in train_responses.itertuples():
            user_id = getattr(row, "query_id")
            item_id = getattr(row, self._base.item_column)
            user_ids.append(user_id)
            item_ids.append(item_id)
            values.append(row.value)
            for epoch in range(num_negatives):
                negative_item_id = rand_state.choice(negative_candidates)
                while used[user_id, negative_item_id] != 0:
                    negative_item_id = rand_state.choice(negative_candidates)

                used[user_id, negative_item_id] = -(epoch + 1)
                user_ids.append(user_id)
                item_ids.append(negative_item_id)
                values.append(negative_value)
    
        return pd.DataFrame({"query_id": user_ids, self._base.item_column: item_ids, "value": values})

    def _compile_dataset(self, query_ids, item_ids, values, batch_size=None):
        X = self._base.compile_feeds(query_ids, item_ids, batch_size=batch_size)
        return BatchData(X=X, Y=values)


class PairwiseData(object):
    def __init__(self, base, random_state=None, batch_size=None):
        LOGGER.critical("PairwiseData revision 20.1.7")
        self._base = base
        self._random_state = sklearn.utils.check_random_state(random_state)
        self._batch_size = batch_size

        self.positive_x = self._base.compile_feeds(self._base.train_responses["query_id"].values,
                                                   self._base.train_responses[self._base.item_column].values,
                                                   batch_size=self._batch_size)
        self.y = np.ones(self._base.train_responses.shape[0])

        # Negative sampling
        self._csr_train_rsvps = scipy.sparse.csr_matrix((self.y,
                                                        (self._base.train_responses["query_id"].values,
                                                         self._base.train_responses[self._base.item_column].values)),
                                                        shape=(self._base.num_queries,
                                                               self._base.feature_counts[self._base.item_column]))
        self._negative_candidates = np.setdiff1d(np.arange(self._base.feature_counts[self._base.item_column]),
                                                 self._base.candidate_item_ids.values)
        self.resample_negatives()

    def resample_negatives(self, random_state=None):
        rand_state = self._random_state if random_state is None else sklearn.utils.check_random_state(random_state)
        used = self._csr_train_rsvps.todok(copy=True)

        users = self._base.train_responses["query_id"].values
        negative_item_ids = list()
        for user_id in users:
            negative_item_id = rand_state.choice(self._negative_candidates)
            while used[user_id, negative_item_id] != 0:
                negative_item_id = rand_state.choice(self._negative_candidates)

            used[user_id, negative_item_id] = -1
            negative_item_ids.append(negative_item_id)

        self.negative_x = self._base.compile_feeds(users, negative_item_ids,
                                                   batch_size=self._batch_size)


class RecommendData(object):
    def __init__(self, base):
        LOGGER.critical("RecommendData revision 19.10.11")
        self._base = base

    def test_query_count(self):
        return self._base.test_query_ids.shape[0]

    def iter_test_query_id_items(self):
        return self._base.test_query_ids.iteritems()

    def iter_candidate_item_ids(self, batch_size):
        for start_index in range(0, self._base.candidate_item_ids.shape[0], batch_size):
            yield self._base.candidate_item_ids.iloc[start_index:(start_index + batch_size)]

    def compile_feeds(self, query_id, item_ids, batch_size=None):
        X = self._base.compile_feeds(query_id, item_ids, batch_size=batch_size)
        return BatchData(X=X, Y=None)


In [13]:
import time

import numpy as np

def recommend(model, data, mb_output_filepath, rank_size=100, batch_size=256):
    """ Generate recommendations from model
    """
    time_begin = time.time()
    # start recommendation
    with open(mb_output_filepath, "wb") as mb_output_file:
        percent_divider = int(data.test_query_count() / 100)
        recommended_queries = 0
        for _, query_id in data.iter_test_query_id_items():
            if recommended_queries % percent_divider == 0:
                LOGGER.info("%d%%: Recommending for query %d",
                            int(recommended_queries / percent_divider), query_id)
            recommended_queries += 1

            y_pred_item_ids = None
            y_pred_values = None
            for item_ids in data.iter_candidate_item_ids(batch_size=batch_size):
                feed_data = data.compile_feeds(query_id, item_ids.values)
                predictions = model.predict(feed_data.X, batch_size=batch_size)

                if y_pred_item_ids is None:
                    y_pred_item_ids = item_ids.values
                    y_pred_values = np.reshape(predictions, (item_ids.shape[0],))
                else:
                    y_pred_item_ids = np.concatenate((y_pred_item_ids, item_ids.values))
                    y_pred_values = np.concatenate((y_pred_values, np.reshape(predictions, (item_ids.shape[0],))))
            
            topK_indices = np.argsort(y_pred_values)[-rank_size:][::-1]
            topK_item_ids = y_pred_item_ids[topK_indices]
            topK_values = y_pred_values[topK_indices]

            ranked_list_str = ','.join(["%d:%f" % id_value for id_value in zip(topK_item_ids, topK_values)])
            mb_output_file.write(("%d\t%s\n" % (query_id, ranked_list_str)).encode("utf-8"))
    LOGGER.info("Time used for recommendation: %.1f s", time.time() - time_begin)


## Running models

In [14]:
def make_model(method, features, embedding_size, learning_rate, *,
               l2_reg=1e-3, dropout_rate=0, init_std=0.01,
               device="cpu"):
    LOGGER.info("%s: embedding_size=%d, learning_rate=%f; l2_reg=%f, dropout_rate=%f, init_std=%f",
                method.upper(), embedding_size, learning_rate, l2_reg, dropout_rate, init_std)
    if method.lower() == "fm":
        model = AFM(linear_feature_columns=features, dnn_feature_columns=features,
                    embedding_size=embedding_size, use_attention=False,
                    l2_reg_linear=l2_reg, l2_reg_embedding=l2_reg, l2_reg_att=l2_reg,
                    init_std=init_std, task="regression", device=device)
    elif method.lower() == "afm":
        model = AFM(linear_feature_columns=features, dnn_feature_columns=features,
                    embedding_size=embedding_size, use_attention=True, attention_factor=embedding_size,
                    l2_reg_linear=l2_reg, l2_reg_embedding=l2_reg, l2_reg_att=l2_reg,
                    afm_dropout=dropout_rate,
                    init_std=init_std, task="regression", device=device)
    elif method.lower() == "nfm":
        model = NFM(linear_feature_columns=features, dnn_feature_columns=features,
                    embedding_size=embedding_size, dnn_hidden_units=(embedding_size, embedding_size),
                    l2_reg_linear=l2_reg, l2_reg_embedding=l2_reg, l2_reg_dnn=l2_reg,
                    bi_dropout=dropout_rate, dnn_dropout=dropout_rate,
                    init_std=init_std, task="regression", device=device)

    model.compile(torch.optim.Adagrad(model.parameters(), lr=learning_rate),
                  loss=None, metrics=["mse", "accuracy"],)

    return model

### Constructing data

In [15]:
import os

dataset_path = os.path.join("/input0", "frappe")
train_feature_file = os.path.join(dataset_path, "deepctr.train.tsv")
test_feature_file = os.path.join(dataset_path, "deepctr.test.tsv")
print(dataset_path, train_feature_file, test_feature_file)

/input0/frappe /input0/frappe/deepctr.train.tsv /input0/frappe/deepctr.test.tsv


In [16]:
base_data = FrappeData(train_feature_file, test_feature_file)

In [17]:
pointwise_train_data = PointwiseData(base_data, negative_value=0, batch_size=4096).train_data

PointwiseData revision 20.1.7


In [18]:
recommend_data = RecommendData(base_data)

RecommendData revision 19.10.11


In [19]:
batch_size = 256
total_epochs = 100
rank_size = 100

### Initializing different models

In [20]:
import torch

# Define Model, train, predict and evaluate
device = 'cpu'
if torch.cuda.is_available():
    LOGGER.critical('cuda ready...')
    device = 'cuda:0'

cuda ready...


In [21]:
# count #unique features for each sparse field
features = list(base_data.features.values())
features

[SparseFeat(name='user', dimension=957, use_hash=False, dtype='int32', embedding_name='user', embedding=True),
 SparseFeat(name='daytime', dimension=7, use_hash=False, dtype='int32', embedding_name='daytime', embedding=True),
 SparseFeat(name='weekday', dimension=7, use_hash=False, dtype='int32', embedding_name='weekday', embedding=True),
 SparseFeat(name='isweekend', dimension=2, use_hash=False, dtype='int32', embedding_name='isweekend', embedding=True),
 SparseFeat(name='homework', dimension=3, use_hash=False, dtype='int32', embedding_name='homework', embedding=True),
 SparseFeat(name='weather', dimension=9, use_hash=False, dtype='int32', embedding_name='weather', embedding=True),
 SparseFeat(name='country', dimension=80, use_hash=False, dtype='int32', embedding_name='country', embedding=True),
 SparseFeat(name='city', dimension=233, use_hash=False, dtype='int32', embedding_name='city', embedding=True),
 SparseFeat(name='item', dimension=4082, use_hash=False, dtype='int32', embedding

#### Standard Factorization Machines

In [ ]:
fm_model = AFM(linear_feature_columns=features, dnn_feature_columns=features,
            embedding_size=256, use_attention=False,
            l2_reg_linear=1e-3, l2_reg_embedding=1e-3, l2_reg_att=1e-3,
            init_std=0.001, task="regression", device=device)

#### Standard Factorization Machines, along with Attentional Factorization Machines with pre-trained embeddings

In [ ]:
for embedding_size in [64, 128, 256]:
    for learning_rate in [0.005, 0.01, 0.02, 0.05]:
        fm_model = make_model("fm", features, embedding_size, learning_rate,
                               device=device)

        # Train model
        pairwise_training_data = PairwiseData(base_data, random_state=2019, batch_size=4096)
        for passing_epoch in range(10, total_epochs + 1, 10):
            pairwise_fit(fm_model, pairwise_training_data, loss_func=regression_pairwise_loss,
                         batch_size=batch_size, epochs=10, verbose=2,
                         validation_data=(pointwise_train_data.X, pointwise_train_data.Y))

            # Recommend from model
            epochs_mb_output_suffix = "F{0}L{1}B256E{2}R001.tsv".format(embedding_size, str(learning_rate)[2:], passing_epoch)
            epochs_mb_output_filepath = "/output/FM_" + epochs_mb_output_suffix
            LOGGER.critical("Saving recommendations to %s", epochs_mb_output_filepath)
            recommend(fm_model, recommend_data, epochs_mb_output_filepath,
                      rank_size=rank_size, batch_size=batch_size)
        
        afm_model = make_model("afm", features, embedding_size, learning_rate,
                               device=device)
        afm_model.embedding_dict = fm_model.embedding_dict

        # Train model
        pairwise_training_data = PairwiseData(base_data, random_state=2019, batch_size=4096)
        for passing_epoch in range(10, total_epochs + 1, 10):
            pairwise_fit(afm_model, pairwise_training_data, loss_func=regression_pairwise_loss,
                         batch_size=batch_size, epochs=10, verbose=2,
                         validation_data=(pointwise_train_data.X, pointwise_train_data.Y))

            # Recommend from model
            epochs_mb_output_suffix = "F{0}L{1}B256E{2}R001.tsv".format(embedding_size, str(learning_rate)[2:], passing_epoch)
            epochs_mb_output_filepath = "/output/AFMpretrained_" + epochs_mb_output_suffix
            LOGGER.critical("Saving recommendations to %s", epochs_mb_output_filepath)
            recommend(afm_model, recommend_data, epochs_mb_output_filepath,
                      rank_size=rank_size, batch_size=batch_size)

FM: embedding_size=64, learning_rate=0.005000; l2_reg=0.001000, dropout_rate=0.000000, init_std=0.010000
PairwiseData revision 20.1.7
cuda:0
Train on 92808 samples, validate on 278424 samples, 363 steps per epoch
Epoch 1/10
13s - loss:  0.1821 - mse:  0.0048 - accuracy:  0.9988 - val_mse:  0.6460 - val_accuracy:  0.3333
Epoch 2/10
10s - loss:  0.1304 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6425 - val_accuracy:  0.3333
Epoch 3/10
9s - loss:  0.1270 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6402 - val_accuracy:  0.3333
Epoch 4/10
10s - loss:  0.1246 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6373 - val_accuracy:  0.3333
Epoch 5/10
9s - loss:  0.1222 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6358 - val_accuracy:  0.3333
Epoch 6/10
10s - loss:  0.1212 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6335 - val_accuracy:  0.3333
Epoch 7/10
8s - loss:  0.1185 - mse:  0.0001 - accuracy:  1.0000 - val_mse:  0.6328 - val_accuracy:  0.3333
Epoch 8/10
8s - loss:  0.11

64%: Recommending for query 17250
65%: Recommending for query 17509
66%: Recommending for query 17615
67%: Recommending for query 17987
68%: Recommending for query 18119
69%: Recommending for query 18358
70%: Recommending for query 18609
71%: Recommending for query 18831
72%: Recommending for query 19294
73%: Recommending for query 19648
74%: Recommending for query 19921
75%: Recommending for query 20374
76%: Recommending for query 20668
77%: Recommending for query 21206
78%: Recommending for query 21295
79%: Recommending for query 21580
80%: Recommending for query 21697
81%: Recommending for query 21961
82%: Recommending for query 22101
83%: Recommending for query 22270
84%: Recommending for query 22579
85%: Recommending for query 22919
86%: Recommending for query 23016
87%: Recommending for query 23401
88%: Recommending for query 24023
89%: Recommending for query 24460
90%: Recommending for query 24950
91%: Recommending for query 25109
92%: Recommending for query 25262
93%: Recommend

28%: Recommending for query 7302
29%: Recommending for query 7465
30%: Recommending for query 8113
31%: Recommending for query 8608
32%: Recommending for query 8801
33%: Recommending for query 9036
34%: Recommending for query 9166
35%: Recommending for query 9244
36%: Recommending for query 9420
37%: Recommending for query 9894
38%: Recommending for query 9969
39%: Recommending for query 10108
40%: Recommending for query 10228
41%: Recommending for query 10415
42%: Recommending for query 10619
43%: Recommending for query 10741
44%: Recommending for query 11429
45%: Recommending for query 11512
46%: Recommending for query 11956
47%: Recommending for query 12165
48%: Recommending for query 12354
49%: Recommending for query 12678
50%: Recommending for query 12912
51%: Recommending for query 13379
52%: Recommending for query 13729
53%: Recommending for query 13848
54%: Recommending for query 14085
55%: Recommending for query 14388
56%: Recommending for query 14699
57%: Recommending for que

Epoch 9/10
10s - loss:  0.0501 - mse:  0.0003 - accuracy:  1.0000 - val_mse:  0.6319 - val_accuracy:  0.3333
Epoch 10/10
10s - loss:  0.0502 - mse:  0.0003 - accuracy:  1.0000 - val_mse:  0.6314 - val_accuracy:  0.3333
Saving recommendations to /output/FM_F64L005B256E60R001.tsv
0%: Recommending for query 16
1%: Recommending for query 330
2%: Recommending for query 429
3%: Recommending for query 550
4%: Recommending for query 741
5%: Recommending for query 1036
6%: Recommending for query 1159
7%: Recommending for query 1369
8%: Recommending for query 1667
9%: Recommending for query 1864
10%: Recommending for query 2300
11%: Recommending for query 2443
12%: Recommending for query 2883
13%: Recommending for query 3086
14%: Recommending for query 3309
15%: Recommending for query 3393
16%: Recommending for query 3730
17%: Recommending for query 4065
18%: Recommending for query 4394
19%: Recommending for query 4576
20%: Recommending for query 4969
21%: Recommending for query 5124
22%: Recomm

96%: Recommending for query 26453
97%: Recommending for query 26707
98%: Recommending for query 26990
99%: Recommending for query 27315
100%: Recommending for query 27877
101%: Recommending for query 28304
102%: Recommending for query 28840
Time used for recommendation: 89.8 s
cuda:0
Train on 92808 samples, validate on 278424 samples, 363 steps per epoch
Epoch 1/10
10s - loss:  0.0467 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6298 - val_accuracy:  0.3333
Epoch 2/10
9s - loss:  0.0461 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6294 - val_accuracy:  0.3333
Epoch 3/10
10s - loss:  0.0460 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6295 - val_accuracy:  0.3333
Epoch 4/10
9s - loss:  0.0459 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6291 - val_accuracy:  0.3333
Epoch 5/10
10s - loss:  0.0451 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6293 - val_accuracy:  0.3333
Epoch 6/10
10s - loss:  0.0447 - mse:  0.0004 - accuracy:  1.0000 - val_mse:  0.6287 - val_accuracy:

...

34%: Recommending for query 9166
35%: Recommending for query 9244
36%: Recommending for query 9420
37%: Recommending for query 9894
38%: Recommending for query 9969
39%: Recommending for query 10108
40%: Recommending for query 10228
41%: Recommending for query 10415
42%: Recommending for query 10619
43%: Recommending for query 10741
44%: Recommending for query 11429
45%: Recommending for query 11512
46%: Recommending for query 11956
47%: Recommending for query 12165
48%: Recommending for query 12354
49%: Recommending for query 12678
50%: Recommending for query 12912
51%: Recommending for query 13379
52%: Recommending for query 13729
53%: Recommending for query 13848
54%: Recommending for query 14085
55%: Recommending for query 14388
56%: Recommending for query 14699
57%: Recommending for query 15108
58%: Recommending for query 15282
59%: Recommending for query 15497
60%: Recommending for query 15576
61%: Recommending for query 15997
62%: Recommending for query 16478
63%: Recommending f

1%: Recommending for query 330
2%: Recommending for query 429
3%: Recommending for query 550
4%: Recommending for query 741
5%: Recommending for query 1036
6%: Recommending for query 1159
7%: Recommending for query 1369
8%: Recommending for query 1667
9%: Recommending for query 1864
10%: Recommending for query 2300
11%: Recommending for query 2443
12%: Recommending for query 2883
13%: Recommending for query 3086
14%: Recommending for query 3309
15%: Recommending for query 3393
16%: Recommending for query 3730
17%: Recommending for query 4065
18%: Recommending for query 4394
19%: Recommending for query 4576
20%: Recommending for query 4969
21%: Recommending for query 5124
22%: Recommending for query 5676
23%: Recommending for query 6050
24%: Recommending for query 6192
25%: Recommending for query 6392
26%: Recommending for query 6785
27%: Recommending for query 7129
28%: Recommending for query 7302
29%: Recommending for query 7465
30%: Recommending for query 8113
31%: Recommending for q

1%: Recommending for query 330
2%: Recommending for query 429
3%: Recommending for query 550
4%: Recommending for query 741
5%: Recommending for query 1036
6%: Recommending for query 1159
7%: Recommending for query 1369
8%: Recommending for query 1667
9%: Recommending for query 1864
10%: Recommending for query 2300
11%: Recommending for query 2443
12%: Recommending for query 2883
13%: Recommending for query 3086
14%: Recommending for query 3309
15%: Recommending for query 3393
16%: Recommending for query 3730
17%: Recommending for query 4065
18%: Recommending for query 4394
19%: Recommending for query 4576
20%: Recommending for query 4969
21%: Recommending for query 5124
22%: Recommending for query 5676
23%: Recommending for query 6050
24%: Recommending for query 6192
25%: Recommending for query 6392
26%: Recommending for query 6785
27%: Recommending for query 7129
28%: Recommending for query 7302
29%: Recommending for query 7465
30%: Recommending for query 8113
31%: Recommending for q

Epoch 1/10
10s - loss:  0.0378 - mse:  0.0041 - accuracy:  1.0000 - val_mse:  0.5380 - val_accuracy:  0.3336
Epoch 2/10
9s - loss:  0.0364 - mse:  0.0042 - accuracy:  1.0000 - val_mse:  0.5374 - val_accuracy:  0.3338
Epoch 3/10
10s - loss:  0.0359 - mse:  0.0042 - accuracy:  1.0000 - val_mse:  0.5359 - val_accuracy:  0.3338
Epoch 4/10
10s - loss:  0.0358 - mse:  0.0043 - accuracy:  1.0000 - val_mse:  0.5351 - val_accuracy:  0.3339
Epoch 5/10
10s - loss:  0.0350 - mse:  0.0044 - accuracy:  1.0000 - val_mse:  0.5330 - val_accuracy:  0.3341
Epoch 6/10
10s - loss:  0.0338 - mse:  0.0045 - accuracy:  1.0000 - val_mse:  0.5327 - val_accuracy:  0.3341
Epoch 7/10
10s - loss:  0.0337 - mse:  0.0046 - accuracy:  1.0000 - val_mse:  0.5316 - val_accuracy:  0.3342
Epoch 8/10
10s - loss:  0.0329 - mse:  0.0047 - accuracy:  1.0000 - val_mse:  0.5307 - val_accuracy:  0.3342
Epoch 9/10
10s - loss:  0.0320 - mse:  0.0047 - accuracy:  1.0000 - val_mse:  0.5318 - val_accuracy:  0.3341
Epoch 10/10
10s - lo

70%: Recommending for query 18609
71%: Recommending for query 18831
72%: Recommending for query 19294
73%: Recommending for query 19648
74%: Recommending for query 19921
75%: Recommending for query 20374
76%: Recommending for query 20668
77%: Recommending for query 21206
78%: Recommending for query 21295
79%: Recommending for query 21580
80%: Recommending for query 21697
81%: Recommending for query 21961
82%: Recommending for query 22101
83%: Recommending for query 22270
84%: Recommending for query 22579
85%: Recommending for query 22919
86%: Recommending for query 23016
87%: Recommending for query 23401
88%: Recommending for query 24023
89%: Recommending for query 24460
90%: Recommending for query 24950
91%: Recommending for query 25109
92%: Recommending for query 25262
93%: Recommending for query 25546
94%: Recommending for query 25826
95%: Recommending for query 26119
96%: Recommending for query 26453
97%: Recommending for query 26707
98%: Recommending for query 26990
99%: Recommend

#### Attentional Factorization Machines

In [ ]:
model = AFM(linear_feature_columns=features, dnn_feature_columns=features,
            embedding_size=256, use_attention=True, attention_factor=256,
            l2_reg_linear=1e-3, l2_reg_embedding=1e-3, l2_reg_att=1e-3,
            afm_dropout=dropout_rate,
            init_std=init_std, task="regression", device=device)

In [ ]:
for embedding_size in [64, 128, 256]:
    for learning_rate in [0.005, 0.01, 0.02, 0.05]:        
        afm_model = make_model("afm", features, embedding_size, learning_rate,
                               device=device)

        # Train model
        pairwise_training_data = PairwiseData(base_data, random_state=2019, batch_size=4096)
        for passing_epoch in range(10, total_epochs + 1, 10):
            pairwise_fit(afm_model, pairwise_training_data, loss_func=regression_pairwise_loss,
                         batch_size=batch_size, epochs=10, verbose=2,
                         validation_data=(pointwise_train_data.X, pointwise_train_data.Y))

            # Recommend from model
            epochs_mb_output_suffix = "F{0}L{1}B256E{2}R001.tsv".format(embedding_size, str(learning_rate)[2:], passing_epoch)
            epochs_mb_output_filepath = "/output/AFM_" + epochs_mb_output_suffix
            LOGGER.critical("Saving recommendations to %s", epochs_mb_output_filepath)
            recommend(afm_model, recommend_data, epochs_mb_output_filepath,
                      rank_size=rank_size, batch_size=batch_size)

#### Neural Factorization Machines

In [ ]:
model = NFM(linear_feature_columns=features, dnn_feature_columns=features,
            embedding_size=256, dnn_hidden_units=(256, 256),
            l2_reg_linear=1e-3, l2_reg_embedding=1e-3, l2_reg_dnn=1e-3,
            bi_dropout=0.2, dnn_dropout=0.2,
            init_std=init_std, task="regression", device=device)

In [ ]:
for embedding_size in [64, 128, 256]:
    for learning_rate in [0.005, 0.01, 0.02, 0.05]:    
        nfm_model = make_model("nfm", features, embedding_size, learning_rate,
                               device=device)

        # Train model
        pairwise_training_data = PairwiseData(base_data, random_state=2019, batch_size=4096)
        for passing_epoch in range(10, total_epochs + 1, 10):
            pairwise_fit(nfm_model, pairwise_training_data, loss_func=regression_pairwise_loss,
                         batch_size=batch_size, epochs=10, verbose=2,
                         validation_data=(pointwise_train_data.X, pointwise_train_data.Y))

            # Recommend from model
            epochs_mb_output_suffix = "F{0}L{1}B256E{2}R001.tsv".format(embedding_size, str(learning_rate)[2:], passing_epoch)
            epochs_mb_output_filepath = "/output/NFM" + epochs_mb_output_suffix
            LOGGER.critical("Saving recommendations to %s", epochs_mb_output_filepath)
            recommend(nfm_model, recommend_data, epochs_mb_output_filepath,
                      rank_size=rank_size, batch_size=batch_size)

### Training models

In [49]:
fm_model.compile(torch.optim.Adagrad(fm_model.parameters(), lr=0.005),
              loss=None, metrics=["mse", "accuracy"],)

In [50]:
pairwise_fit(fm_model, pairwise_training_data, loss_func=regression_pairwise_loss,
             batch_size=256, epochs=10, verbose=2,
             validation_data=(pointwise_train_data.X, pointwise_train_data.Y))

cuda:0
Train on 92808 samples, validate on 278424 samples, 363 steps per epoch
Epoch 1/10
10s - loss:  0.1667 - mse:  0.0036 - accuracy:  0.9980 - val_mse:  0.5585 - val_accuracy:  0.3333
Epoch 2/10
10s - loss:  0.1216 - mse:  0.0021 - accuracy:  1.0000 - val_mse:  0.5430 - val_accuracy:  0.3333
Epoch 3/10
10s - loss:  0.1069 - mse:  0.0025 - accuracy:  1.0000 - val_mse:  0.5414 - val_accuracy:  0.3333
Epoch 4/10
10s - loss:  0.0945 - mse:  0.0026 - accuracy:  1.0000 - val_mse:  0.5452 - val_accuracy:  0.3333
Epoch 5/10
11s - loss:  0.0848 - mse:  0.0027 - accuracy:  1.0000 - val_mse:  0.5487 - val_accuracy:  0.3333
Epoch 6/10
9s - loss:  0.0775 - mse:  0.0027 - accuracy:  1.0000 - val_mse:  0.5481 - val_accuracy:  0.3333
Epoch 7/10
10s - loss:  0.0719 - mse:  0.0028 - accuracy:  1.0000 - val_mse:  0.5511 - val_accuracy:  0.3333
Epoch 8/10
9s - loss:  0.0679 - mse:  0.0028 - accuracy:  1.0000 - val_mse:  0.5522 - val_accuracy:  0.3333
Epoch 9/10
10s - loss:  0.0642 - mse:  0.0029 - acc

### Produce recommendations

In [ ]:
recommend(model, recommend_data, "/output/FM_F256L", rank_size=rank_size)